In [ ]:
%pip install beautifulsoup4

In [ ]:
%pip install --upgrade beautifulsoup4

In [ ]:
%pip install lxml

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url1 = "https://seekingalpha.com/symbol/YPF/news"

# Definir el User-Agent válido
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Realizar la solicitud HTTP con el User-Agent definido
response = requests.get(url1, headers=headers)

response

<Response [403]>

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="px-captcha" name="description"/><title>Access to this page has been denied</title></head><body><script>window._pxVid='';window._pxUuid='0333a94a-f6e9-11ee-a844-0bc8348a0026';window._pxAppId='PXxgCxM9By';window._pxHostUrl='/xgCxM9By/xhr';window._pxCustomLogo='';window._pxJsClientSrc='/xgCxM9By/init.js';window._pxMobile=false;window._pxFirstPartyEnabled=true;var pxCaptchaSrc='/xgCxM9By/captcha/PXxgCxM9By/captcha.js?a=c&u=0333a94a-f6e9-11ee-a844-0bc8348a0026&v=&m=0&b=aHR0cHM6Ly9zc3Iuc2Vla2luZ2FscGhhLmNvbS9zeW1ib2wvWVBGL25ld3M=';var script=document.createElement('script');script.src=pxCaptchaSrc;script.onload=onScriptLoad;script.onerror=onScriptError;var onScriptErrorCalled;document.head.appendChild(script);var timeoutID=setTimeout(onScriptError,5000);function onScriptLoad(){clearTimeout(timeoutID);setTimeout(function(){if (isCaptchaNotLoaded()){on

In [84]:
# Utiliza un selector CSS para encontrar las etiquetas article con la clase especificada
seeking = soup.select("article.flex")

seeking_news = []

# Itera sobre las etiquetas article
for i in seeking:
    # Obtiene el título del artículo
    title = i.find("a").text.strip()

    # Busca el elemento que contiene la fecha
    date_element = i.find("span", {"data-test-id": "post-list-date"})
    
    # Extrae solo el texto de la fecha
    date_text = date_element.text.strip() 

    # Crear un diccionario con el título y la fecha y agregarlo a la lista de noticias buscadas
    news_item = {"titulo": title, "fecha": date_text}
    seeking_news.append(news_item)

print(seeking_news)

[{'titulo': "Golar LNG surges as FLNG Hilli said to be headed for Argentina's Vaca Muerta", 'fecha': 'Tue, Mar. 12'}, {'titulo': 'YPF swings to Q4 net loss, plans $3B in shale spending for 2024', 'fecha': 'Thu, Mar. 07'}, {'titulo': 'Exxon weighing $1B sale of Argentina shale assets - Bloomberg', 'fecha': 'Fri, Feb. 23'}, {'titulo': "YPF upgraded to Buy at UBS, seeing Petrobras' recent gains as benchmark", 'fecha': 'Thu, Feb. 22'}, {'titulo': 'Energy stocks score best week since March as oil prices ride Middle East tensions', 'fecha': 'Fri, Jan. 26'}, {'titulo': 'YPF shoots higher after Citi upgrade to Buy following recent pullback', 'fecha': 'Thu, Jan. 25'}, {'titulo': 'YPF privatization omitted from Argentina government omnibus bill', 'fecha': 'Mon, Jan. 22'}, {'titulo': 'Burford Capital jumps on report Argentina considers bond to pay for $16B judgement (update)', 'fecha': 'Wed, Dec. 27, 2023'}, {'titulo': "Argentina's Milei gets the ball rolling on his privatization drive", 'fecha':

In [4]:
## Mod de Fer

import time
import random
import pandas as pd

def scrape_data(url):
    results = []

    page = 1
    while True:
        page_url = f"{url}{page}"
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
        response = requests.get(page_url, headers=headers)
        
        time.sleep(random.randint(5, 15))
        
        if response.status_code != 200: #Esto lo hago porque sino Tamara me reta :)
            print(f"Error 200 en página {page}.")
            break

        soup = BeautifulSoup(response.content, 'html.parser')

        items = soup.select("article.flex")

        if len(items)==0 or items is None:
            print(f"La página {page} no existe o no contiente datos.")
            break

        for item in items:
            date_span = item.find("span", {"data-test-id": "post-list-date"}).text.strip() 
            title = item.find("a").text.strip()

            result = {
                "Fecha": date_span,
                "Titulo": title,
                "Fuente": "Seeking Alpha"
            }
            results.append(result)

        print(f"Página {page} procesada")
        page += 1
        time.sleep(random.randint(10, 15))

    return results

In [5]:
url_base = "https://seekingalpha.com/symbol/YPF/news?page="

resultados = scrape_data(url_base)
df_resultados = pd.DataFrame(resultados)

Error 200 en página 1.


### Formato

In [20]:
from datetime import datetime

df_resultados_fecha = df_resultados

df_resultados_fecha['Fecha'] = df_resultados['Fecha'].apply(lambda x: datetime.strptime(x, "%a, %b. %d, %Y").strftime("%d/%m/%Y"))

KeyError: 'Fecha'

In [ ]:
df_resultados_fecha.to_excel('Scraping_SeekingAlpha.xlsx', index=False)